In [106]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import pmdarima as pm
from pmdarima.utils import diff
from sklearn.metrics import mean_squared_error
from pmdarima.metrics import smape
import pickle

In [107]:
df = pd.read_csv('./../data/zri_multifamily_v2.csv')

In [108]:
df['zip'] = df['zip'].map(lambda x: str(x))

In [109]:
df['zip'] = df['zip'].map(lambda x: '0' + x if len(x)<5 else x)

In [110]:
df['datetime']=df['year-month'].map(lambda x: datetime.strptime(x, '%Y-%m'))

In [111]:
df2=df.set_index('datetime')

In [112]:
df2=df2[['zri', 'zip']]

In [113]:
train = df2[df2.index <= '2018-12']

In [114]:
test = df2[df2.index > '2018-12']

In [115]:
ytrain = pd.concat([train['zip'], train['zri']], axis = 1)

In [116]:
zips = ytrain['zip'].unique()

In [117]:
len(zips)

1302

In [118]:
arima = pm.arima.auto_arima

In [119]:
models_dict = {}
fc_dict = {}

In [120]:
def forecast_one_step():
    fc, conf_int = curr_model.predict(n_periods=1, return_conf_int=True)
    return (
        fc.tolist()[0],
        np.asarray(conf_int).tolist()[0])

In [121]:
start = datetime.now()
k=0
for item in zips:
    curr_model = arima(train.loc[train['zip']==item,'zri'], start_p=0, start_q=0, m=12)

    true = test[test['zip']==item]
    pred = []
    confidence_intervals = []

    for new_ob in true['zri']:
        fc, conf = forecast_one_step()
        pred.append(fc)
        confidence_intervals.append(conf)

        # Updates the existing model with the prediction it just made
        curr_model.update(fc)    
    
    rsme=(sum((true['zri']-pred)**2)/len(pred))**0.5
    
    models_dict[item] = (curr_model.order, curr_model.seasonal_order, rsme)
    fc_dict[item] = (pred, confidence_intervals)
    
    elapsed = datetime.now() - start
    k=k+1
    print(elapsed)
    print(k)
    print(rsme)

0:00:01.071116
1
43.80699862203557
0:00:02.435368
2
39.48391329694133
0:00:03.119450
3
42.71219073964782
0:00:05.161879
4
29.546903869601607
0:00:06.521041
5
42.18611426513878
0:00:09.730259
6
28.780146424942004
0:00:10.937421
7
28.708078048894468
0:00:25.783319
8
24.048793355471755
0:00:29.121427
9
57.4310040411475
0:00:29.751232
10
41.15533571880509
0:00:31.110747
11
57.90460737225107
0:00:31.789392
12
56.268502687695495
0:00:32.890084
13
32.83266040020293
0:00:33.484326
14
37.38323075813117
0:00:34.843711
15
22.220706331277476
0:00:35.449314
16
9.989120482167376
0:00:36.454870
17
60.265018935796334
0:00:38.441230
18
103.80634268175096
0:00:39.866007
19
57.194254413596845
0:00:41.590124
20
47.32357668956139
0:00:43.924165
21
139.64214384282837
0:00:47.976890
22
36.803186228637124
0:00:49.957273
23
13.157479415832908
0:00:52.933775
24
12.314579004545646
0:00:59.179769
25
63.34143146834948
0:01:07.417745
26
47.468151913844466
0:01:11.936516
27
52.42666327272829
0:01:14.678397
28
58.918

0:10:43.624941
224
125.24830525846411
0:10:44.536327
225
129.52913187014352
0:10:45.339582
226
52.56355565039052
0:10:46.386983
227
134.84059962619077
0:10:49.121104
228
89.71886291559139
0:10:50.697676
229
47.74660751024886
0:10:52.283770
230
55.19397815615912
0:10:53.519093
231
68.70443212906638
0:10:59.567686
232
17.46534675247572
0:11:00.864373
233
51.7167019922212
0:11:02.509442
234
42.4731383533298
0:11:03.794223
235
39.06192540679153
0:11:08.365054
236
63.05258198473185
0:11:12.722504
237
39.52672336419102
0:11:13.480557
238
58.26957805782132
0:11:14.218759
239
25.78829471943025
0:11:17.774948
240
17.369842446320327
0:11:18.472291
241
8.781718535780696
0:11:19.269351
242
41.72720880413127
0:11:19.704883
243
27.781556000202052
0:11:20.552095
244
10.215476871317902
0:11:21.290159
245
23.699368460200272
0:11:24.791633
246
18.419224841816337
0:11:25.670720
247
35.00624944206391
0:11:27.730024
248
14.234529213576565
0:11:30.400166
249
53.33621713163606
0:11:32.336756
250
29.113152191

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(3,0,0)(0,0,1)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/ml

0:16:12.688902
397
31.24104467473078
0:16:14.572884
398
44.49237157178493
0:16:18.222015
399
22.760724487658084
0:16:18.990630
400
39.398469941667834
0:16:20.036411
401
25.04770301705304
0:16:22.680146
402
16.58319001554205
0:16:24.108685
403
11.350064665401078
0:16:24.735972
404
25.50736262329057
0:16:28.150671
405
44.646191735928156
0:16:29.866003
406
18.92516805302499
0:16:32.382459
407
16.564471489355494
0:16:35.730740
408
14.347303300877824
0:16:36.442827
409
18.430322291017216
0:16:38.376730
410
10.329522997804679
0:16:39.150228
411
13.30795485085465
0:16:43.103756
412
16.716340472809843
0:16:45.291938
413
15.890012245288638
0:16:50.193813
414
42.32200493085912
0:16:51.243095
415
18.241728525301042
0:16:52.727712
416
16.833085292992248
0:16:54.185115
417
12.01284463274694
0:17:02.414639
418
21.75322867626931
0:17:05.030391
419
10.312076299758195
0:17:05.604025
420
68.5527025946644
0:17:07.474055
421
11.346475539107818
0:17:09.058258
422
13.986906827773513
0:17:10.053868
423
13.33

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,1,1)(0,0,1)[12] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/ml

...

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,0)(2,0,1)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/ml

0:36:22.568557
944
25.661527243207473
0:36:23.478031
945
18.477828333714456
0:36:25.215492
946
42.52425494552979
0:36:26.034397
947
35.99427434186749


/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py:522: ModelFitWarning: Error fitting  ARIMA(2,0,0)(0,0,0)[12]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py", line 506, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 482, in fit
    self._fit(y, X, **fit_args)
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 403, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 393, in _fit_wrapper
    return arima, arima.fit(start_params=start_params,
  File "/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/ml

0:36:28.747523
948
33.2903786180809
0:36:30.651136
949
22.556768000785105
0:36:31.242611
950
25.98668155303903
0:36:32.211177
951
62.57998377238128
0:36:33.873029
952
66.29978512053306
0:36:35.734342
953
36.08517446593984
0:36:36.515189
954
9.182397732799563
0:36:38.782588
955
27.241562223988986
0:36:41.814822
956
30.566094042304986
0:36:43.566322
957
13.179516905332523
0:36:44.672534
958
18.423119063213928
0:36:47.884821
959
89.06866490881944
0:36:49.712305
960
54.60906920127602
0:36:50.413609
961
47.98654533543932
0:36:57.138555
962
13.697268883832292
0:37:01.698966
963
53.837208390971064
0:37:03.226230
964
59.632183463167834
0:37:04.239179
965
21.812720907605534
0:37:11.319612
966
21.594339343772653
0:37:12.143810
967
60.06736547725144
0:37:13.613742
968
10.392333157482309
0:37:16.953936
969
57.86600191139145
0:37:19.648969
970
47.22059938565068
0:37:20.882333
971
34.12343519739478
0:37:21.527447
972
23.089542817267258
0:37:22.461396
973
14.429475641671837
0:37:23.722284
974
65.9758

0:42:42.596701
1163
55.638327663414785
0:42:44.566508
1164
58.631657537804756
0:42:46.550860
1165
51.84713703986002
0:42:48.276049
1166
25.617844615010693
0:42:48.959005
1167
36.38047737634318
0:42:49.989629
1168
23.868607199202362
0:42:54.614566
1169
36.371136132921805
0:42:56.156489
1170
66.22998591251341
0:42:58.097450
1171
90.23085273301031
0:42:59.972134
1172
42.21779566370456
0:43:01.681291
1173
72.60683890640772
0:43:06.009245
1174
57.70926758246627
0:43:06.686147
1175
54.97853425105751
0:43:07.428767
1176
38.81106029484507
0:43:08.487948
1177
27.09096266259901
0:43:09.119814
1178
81.17482812604219
0:43:09.943469
1179
53.956321227459036
0:43:12.002628
1180
57.358064988794915
0:43:12.863136
1181
37.481106351511734
0:43:31.379042
1182
25.241269883261033
0:43:33.900317
1183
145.2566378201705
0:43:34.563425
1184
68.17823074440416
0:43:35.047870
1185
13.915569353959695
0:43:35.724377
1186
26.868710952887554
0:43:36.782121
1187
164.33629178836273
0:43:37.532030
1188
36.448021284439186

In [122]:
curr_model

ARIMA(order=(0, 1, 1), scoring_args={}, seasonal_order=(1, 0, 0, 12),
      suppress_warnings=True, with_intercept=False)

In [123]:
models_dict2=models_dict.copy()
x=0
for model in models_dict2.values():
    x+=model[2]
    
x=x/len(models_dict2)
x

50.25465387264108

In [124]:
models_dict2=models_dict.copy()

zipcode=[]
for model in models_dict2.keys():
    zipcode.append(model)

arima_params=[]
for model in models_dict2.values():
    arima_params.append(model[0])
    
arima_seasonal=[]
for model in models_dict2.values():
    arima_seasonal.append(model[1])
    
rsme=[]
for model in models_dict2.values():
    rsme.append(model[2])

pred=[]
for model in fc_dict.values():
    pred.append(model[0])
    
confidence_intervals=[]
for model in fc_dict.values():
    confidence_intervals.append(model[1])
    
    
    
arima_results=pd.DataFrame({'ZIP':zipcode, 'arima_params':arima_params,'arima_seasonal':arima_seasonal,'RSME':rsme,'Predictions_2019':pred,'Conf_interval_2019':confidence_intervals})


In [125]:
arima_results

,ZIP,arima_params,arima_seasonal,RSME,Predictions_2019,Conf_interval_2019
0,01013,"(1, 1, 0)","(0, 0, 0, 12)",43.806999,"[1104.9205922703397, 1102.1462652838563, 1100....","[[1093.3254754811594, 1116.51570905952], [1090..."
1,01020,"(1, 1, 0)","(0, 0, 1, 12)",39.483913,"[1127.026942368522, 1128.0245225972649, 1131.0...","[[1116.9905656725252, 1137.0633190645187], [11..."
2,01040,"(1, 1, 0)","(0, 0, 0, 12)",42.712191,"[1096.8463435276497, 1094.689110210824, 1093.5...","[[1081.8178964641788, 1111.8747905911205], [10..."
3,01085,"(1, 1, 0)","(1, 0, 0, 12)",29.546904,"[1158.3256468861262, 1156.1719939932464, 1157....","[[1140.7779237720952, 1175.8733700001571], [11..."
4,01104,"(2, 1, 0)","(0, 0, 0, 12)",42.186114,"[1113.8984539135715, 1112.6254001848522, 1112....","[[1102.787951903449, 1125.008955923694], [1101..."
...,...,...,...,...,...,...
1297,99207,"(2, 1, 0)","(0, 0, 0, 12)",25.156877,"[1036.6481588694755, 1042.1147839616526, 1045....","[[1020.0944291858186, 1053.2018885531324], [10..."
1298,99501,"(2, 1, 0)","(0, 0, 0, 12)",28.482952,"[1268.7548078416824, 1265.8719548836218, 1261....","[[1257.661694913128, 1279.8479207702367], [125..."
1299,99504,"(2, 1, 1)","(0, 0, 0, 12)",24.028092,"[1442.0497054601437, 1446.44050379072, 1448.47...","[[1426.9500081407898, 1457.1494027794977], [14..."
1300,99508,"(2, 1, 1)","(0, 0, 0, 12)",18.313647,"[1283.450675221121, 1285.3850811051711, 1284.8...","[[1271.371847456103, 1295.5295029861388], [127..."


In [126]:
arima_results.to_csv('./../data/arima_model_results.csv')